In [1]:
import pandas as pd
pd.options.display.max_columns = 999
pd.options.display.max_rows = 30
import numpy as np


In [2]:
[]##Define the object for the season team
class MyDf:
    def __init__(self, mydf):
        self.df = mydf   

# Creating the teams dataframe

In [836]:
TEAMNAMES=pd.read_csv('../Data/from_sqlite_Data/Team.csv')
LEAGUENAMES= pd.read_csv('../Data/from_sqlite_Data/League.csv')
df2 = pd.read_csv('../Data/from_sqlite_Data/bigfinal.csv')
df2.drop(['Unnamed: 0'],axis=1,inplace=True)
df2.drop(['league_id'],axis=1,inplace=True)
df2.drop(['id'],axis=1,inplace=True)

In [837]:
season = df2
FTR=[]
for i in range(len(season)):
    if season['FTHG'][i]>season['FTAG'][i]:
        FTR.append('H')
    if season['FTHG'][i]<season['FTAG'][i]:
        FTR.append('A')
    if season['FTHG'][i]==season['FTAG'][i]:
        FTR.append('D')
    
season['FTR']=FTR   

In [838]:
season=season.sort('Date')

/Users/nicolamacchitella/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [839]:
season.reset_index(inplace=True)


In [840]:
season.drop(['index'],axis=1,inplace=True)

In [841]:
#season['season']=['2016/2017']*len(season)

In [842]:
teams=season.HomeTeam.unique()
allteamsresults=[]
for i in teams:
    path=pd.DataFrame()            
    path = season[(season["HomeTeam"] == i) | (season['AwayTeam'] == i )] 
    allteamsresults.append(MyDf(path))

In [3]:
def point_in_round_k(ALL_TEAM,i,team_index,where):
    
    home=where
    away= not home
    
    if home:
        if ALL_TEAM[team_index].df['FTR'][i]=='H':
            return 3
        if ALL_TEAM[team_index].df['FTR'][i]=='D':
            return 1
        if ALL_TEAM[team_index].df['FTR'][i]=='A':
            return 0
    #   
    if away:
        if ALL_TEAM[team_index].df['FTR'][i]=='H':
            return 0
        if ALL_TEAM[team_index].df['FTR'][i]=='D':
            return 1
        if ALL_TEAM[team_index].df['FTR'][i]=='A':
            return 3

In [4]:
def zero_or_big(a,b):
    if a-b>0:
        return a-b
    return 0

In [5]:
def poin_in_last_n_mathces(ALL_TEAM,i,team_index,where,round_index,n):
    if round_index<=n:
        return point_in_round_k(ALL_TEAM,i,team_index,where)
        

## Allteamsresults is a list of dataframes and each dataframe represents a specific team. To access info on a team use team index. The position of the team in the list "teams" gives  the index of the team in the "allteamsresults" list of dataframes.

## Now, lets calculate some other features

## 1) Goal scored and received - home and away.
### We will shift of one match back each calculation, which means, each row will have the present match and the statistics goals scored up to that day (before the match).

In [6]:
def calculate_for_team_i(ALL_TEAM,team_lists,team_index):
    
    #create the lists for goals 
    Goal_scored_home=[]
    Goal_received_home=[]
    Goal_scored_away=[]
    Goal_received_away=[]
    
    #total_goal
    total_goal_scored=[]
    total_goal_received=[]
    
    #create the list for averages goals
    Av_Goal_scored_home=[]
    Av_Goal_received_home=[]
   

    Av_Goal_scored_away=[]
    Av_Goal_received_away=[]
       
    #create the list for averages total goals
    Av_total_goal_scored=[]
    Av_total_goal_received=[]
    
    #create the lists for points
    Point_home=[]
    Point_away=[]

    total_point=[]

    #create the list for average Points
    Av_Point_home=[]
    Av_Point_away=[]

    Av_total_point=[]
    
    
    #Point in the last mathces
    
    T_L3M_Pnt=[]
    T_L5M_Pnt=[]
    
    H_L3M_Pnt=[]
    A_L3M_Pnt=[]
    
    H_L5M_Pnt=[]
    A_L5M_Pnt=[]
    
    
    
    #ROUND_INDEX
    round_index=[]

    
    k=1
    
    Goal_scored_home.append(0)
    Goal_received_home.append(0)
    Goal_scored_away.append(0)
    Goal_received_away.append(0)
    total_goal_scored.append(0)
    total_goal_received.append(0)
    Av_Goal_scored_home.append(0)
    Av_Goal_received_home.append(0)
    Av_Goal_scored_away.append(0)
    Av_Goal_received_away.append(0)
    Av_total_goal_scored.append(0)
    Av_total_goal_received.append(0)
    Point_home.append(0)
    Point_away.append(0)
    total_point.append(0)
    Av_Point_home.append(0)
    Av_Point_away.append(0)
    Av_total_point.append(0) 
    T_L3M_Pnt.append(0)
    T_L5M_Pnt.append(0)
    #H_L3M_Pnt.append(0)
    #A_L3M_Pnt.append(0)
    #H_L5M_Pnt.append(0)
    #A_L5M_Pnt.append(0)
    
    
    for h in allteamsresults[team_index].df['season'].unique():
    
        seasonal_index=allteamsresults[team_index].df[allteamsresults[team_index].df['season']==h].index
        Home=0
        Away=0
        total=0
        round_in=1
        tmp3=[]
        tmp5=[]
        
        for i in seasonal_index:
            
            round_index.append(round_in)
            round_index_2=round_in-1
            
            home_1=ALL_TEAM[team_index].df['HomeTeam'][i]==team_lists[team_index]
            away_1= not home_1
            

            if (round_in<=3):
                if home_1:
                    if ALL_TEAM[team_index].df['FTR'][i]=='H':
                        T_L3M_Pnt.append(3)
                        tmp3.append(3)
                        #H_L3M_Pnt.append(3)
                        #A_L3M_Pnt.append(A_L3M_Pnt[k-1])
                    if ALL_TEAM[team_index].df['FTR'][i]=='D':
                        T_L3M_Pnt.append(1) 
                        tmp3.append(1)                        
                        #H_L3M_Pnt.append(1)
                        #A_L3M_Pnt.append(A_L3M_Pnt[k-1])
                    if ALL_TEAM[team_index].df['FTR'][i]=='A':
                        T_L3M_Pnt.append(0)                        
                        tmp3.append(0)
                        #H_L3M_Pnt.append(0)
                        #A_L3M_Pnt.append(A_L3M_Pnt[k-1])
                #   
                if away_1:
                    if ALL_TEAM[team_index].df['FTR'][i]=='H':
                        T_L3M_Pnt.append(0)
                        tmp3.append(0)
                        #H_L3M_Pnt.append(H_L3M_Pnt[k-1])
                        #A_L3M_Pnt.append(0)
                    if ALL_TEAM[team_index].df['FTR'][i]=='D':
                        T_L3M_Pnt.append(1)
                        tmp3.append(1)
                        #H_L3M_Pnt.append(H_L3M_Pnt[k-1])
                        #A_L3M_Pnt.append(1)
                    if ALL_TEAM[team_index].df['FTR'][i]=='A':
                        T_L3M_Pnt.append(3)
                        tmp3.append(3)
                        #H_L3M_Pnt.append(H_L3M_Pnt[k-1])
                        #A_L3M_Pnt.append(3)
            #                        
            if (round_in<=5):
                if home_1:
                    if ALL_TEAM[team_index].df['FTR'][i]=='H':
                        T_L5M_Pnt.append(3)
                        tmp5.append(3)
                        #H_L5M_Pnt.append(3)
                        #A_L5M_Pnt.append(A_L5M_Pnt[k-1])

                    if ALL_TEAM[team_index].df['FTR'][i]=='D':
                        T_L5M_Pnt.append(1)
                        tmp5.append(1)
                        #H_L5M_Pnt.append(1)
                        #A_L5M_Pnt.append(A_L5M_Pnt[k-1])

                    if ALL_TEAM[team_index].df['FTR'][i]=='A':
                        T_L5M_Pnt.append(0)
                        tmp5.append(0)
                        #H_L5M_Pnt.append(0)
                        #A_L5M_Pnt.append(A_L5M_Pnt[k-1])

                if away_1:
                    if ALL_TEAM[team_index].df['FTR'][i]=='H':
                        T_L5M_Pnt.append(0)
                        tmp5.append(0)
                        #H_L5M_Pnt.append(H_L5M_Pnt[k-1])
                        #A_L5M_Pnt.append(0)

                    if ALL_TEAM[team_index].df['FTR'][i]=='D':
                        T_L5M_Pnt.append(1)
                        tmp5.append(1)
                        #H_L5M_Pnt.append(H_L5M_Pnt[k-1])
                        #A_L5M_Pnt.append(1)

                    if ALL_TEAM[team_index].df['FTR'][i]=='A':
                        T_L5M_Pnt.append(3)
                        tmp5.append(3)
                        #H_L5M_Pnt.append(H_L5M_Pnt[k-1])
                        #A_L5M_Pnt.append(3)
                        
            '''if (round_in==3):
                if home_1:
                    if ALL_TEAM[team_index].df['FTR'][i]=='H':
                        T_L3M_Pnt.append(3++T_L3M_Pnt[round_index_2-1]+T_L3M_Pnt[round_index_2-2])
                        #H_L3M_Pnt.append(3)
                        #A_L3M_Pnt.append(A_L3M_Pnt[k-1])
                    if ALL_TEAM[team_index].df['FTR'][i]=='D':
                        T_L3M_Pnt.append(1++T_L3M_Pnt[round_index_2-1]+T_L3M_Pnt[round_index_2-2])                        
                        #H_L3M_Pnt.append(1)
                        #A_L3M_Pnt.append(A_L3M_Pnt[k-1])
                    if ALL_TEAM[team_index].df['FTR'][i]=='A':
                        T_L3M_Pnt.append(0++T_L3M_Pnt[round_index_2-1]+T_L3M_Pnt[round_index_2-2])                        
                        #H_L3M_Pnt.append(0)
                        #A_L3M_Pnt.append(A_L3M_Pnt[k-1])
                #   
                #
                if away_1:
                    if ALL_TEAM[team_index].df['FTR'][i]=='H':
                        T_L3M_Pnt.append(0++T_L3M_Pnt[round_index_2-1]+T_L3M_Pnt[round_index_2-2])
                        #H_L3M_Pnt.append(H_L3M_Pnt[k-1])
                        #A_L3M_Pnt.append(0)
                    if ALL_TEAM[team_index].df['FTR'][i]=='D':
                        T_L3M_Pnt.append(1+T_L3M_Pnt[round_index_2-1]+T_L3M_Pnt[round_index_2-2])
                        #H_L3M_Pnt.append(H_L3M_Pnt[k-1])
                        #A_L3M_Pnt.append(1)
                    if ALL_TEAM[team_index].df['FTR'][i]=='A':
                        T_L3M_Pnt.append(3++T_L3M_Pnt[round_index_2-1]+T_L3M_Pnt[round_index_2-2])
                        #H_L3M_Pnt.append(H_L3M_Pnt[k-1])
                        #A_L3M_Pnt.append(3)
            
            if (round_in==5):
                if home_1:
                    if ALL_TEAM[team_index].df['FTR'][i]=='H':
                        T_L5M_Pnt.append(3+T_L5M_Pnt[round_index_2-1]+T_L5M_Pnt[round_index_2-2]+T_L5M_Pnt[round_index_2-3]
                                        +T_L5M_Pnt[round_index_2-4])
                        #H_L5M_Pnt.append(3)
                        #A_L5M_Pnt.append(A_L5M_Pnt[k-1])

                    if ALL_TEAM[team_index].df['FTR'][i]=='D':
                        T_L5M_Pnt.append(1+T_L5M_Pnt[round_index_2-1]+T_L5M_Pnt[round_index_2-2]+T_L5M_Pnt[round_index_2-3]
                                        +T_L5M_Pnt[round_index_2-4])
                        #H_L5M_Pnt.append(1)
                        #A_L5M_Pnt.append(A_L5M_Pnt[k-1])

                    if ALL_TEAM[team_index].df['FTR'][i]=='A':
                        T_L5M_Pnt.append(0+T_L5M_Pnt[round_index_2-1]+T_L5M_Pnt[round_index_2-2]+T_L5M_Pnt[round_index_2-3]
                                        +T_L5M_Pnt[round_index_2-4])
                        #H_L5M_Pnt.append(0)
                        #A_L5M_Pnt.append(A_L5M_Pnt[k-1])
                if away_1:
                    if ALL_TEAM[team_index].df['FTR'][i]=='H':
                        T_L5M_Pnt.append(0+T_L5M_Pnt[round_index_2-1]+T_L5M_Pnt[round_index_2-2]+T_L5M_Pnt[round_index_2-3]
                                        +T_L5M_Pnt[round_index_2-4])
                        #H_L5M_Pnt.append(H_L5M_Pnt[k-1])
                        #A_L5M_Pnt.append(0)

                    if ALL_TEAM[team_index].df['FTR'][i]=='D':
                        T_L5M_Pnt.append(1+T_L5M_Pnt[round_index_2-1]+T_L5M_Pnt[round_index_2-2]+T_L5M_Pnt[round_index_2-3]
                                        +T_L5M_Pnt[round_index_2-4])
                        #H_L5M_Pnt.append(H_L5M_Pnt[k-1])
                        #A_L5M_Pnt.append(1)

                    if ALL_TEAM[team_index].df['FTR'][i]=='A':
                        T_L5M_Pnt.append(3+T_L5M_Pnt[round_index_2-1]+T_L5M_Pnt[round_index_2-2]+T_L5M_Pnt[round_index_2-3]
                                        +T_L5M_Pnt[round_index_2-4])
                        #H_L5M_Pnt.append(H_L5M_Pnt[k-1])
                        #A_L5M_Pnt.append(3)
'''
            if (round_in>3):
                if home_1:
                    if ALL_TEAM[team_index].df['FTR'][i]=='H':
                        tmp3.append(3)
                        del tmp3[0]
                        T_L3M_Pnt.append(sum(tmp3))

                    if ALL_TEAM[team_index].df['FTR'][i]=='D':  
                        tmp3.append(1)
                        del tmp3[0]
                        T_L3M_Pnt.append(sum(tmp3))
                    
                    if ALL_TEAM[team_index].df['FTR'][i]=='A':
                        tmp3.append(0)
                        del tmp3[0]
                        T_L3M_Pnt.append(sum(tmp3))
                        
                if away_1:
                    if ALL_TEAM[team_index].df['FTR'][i]=='H':
                        tmp3.append(0)
                        del tmp3[0]
                        T_L3M_Pnt.append(sum(tmp3))
                        
                    if ALL_TEAM[team_index].df['FTR'][i]=='D':  
                        tmp3.append(1)
                        del tmp3[0]
                        T_L3M_Pnt.append(sum(tmp3))
                        
                        
                    if ALL_TEAM[team_index].df['FTR'][i]=='A':
                        tmp3.append(3)
                        del tmp3[0]
                        T_L3M_Pnt.append(sum(tmp3))
                        
                        
            if (round_in>5):
                if home_1:
                    if ALL_TEAM[team_index].df['FTR'][i]=='H':
                        tmp5.append(3)
                        del tmp5[0]
                        T_L5M_Pnt.append(sum(tmp5))
                        
                    if ALL_TEAM[team_index].df['FTR'][i]=='D':
                        tmp5.append(1)
                        del tmp5[0]
                        T_L5M_Pnt.append(sum(tmp5))
                        
                    if ALL_TEAM[team_index].df['FTR'][i]=='A':
                        tmp5.append(0)
                        del tmp5[0]
                        T_L5M_Pnt.append(sum(tmp5))
                        
                if away_1:
                    if ALL_TEAM[team_index].df['FTR'][i]=='H':
                        tmp5.append(0)
                        del tmp5[0]
                        T_L5M_Pnt.append(sum(tmp5))
                        
                    if ALL_TEAM[team_index].df['FTR'][i]=='D':
                        tmp5.append(1)
                        del tmp5[0]
                        T_L5M_Pnt.append(sum(tmp5))
                        
                    if ALL_TEAM[team_index].df['FTR'][i]=='A':
                        tmp5.append(3)
                        del tmp5[0]
                        T_L5M_Pnt.append(sum(tmp5))
            
            round_in=round_in+1
            

            #HOME
            if ALL_TEAM[team_index].df['HomeTeam'][i]==team_lists[team_index]:

                #Goal scored, received and total
                Goal_scored_home.append(ALL_TEAM[team_index].df['FTHG'][i]+Goal_scored_home[k-1])
                Goal_received_home.append(ALL_TEAM[team_index].df['FTAG'][i]+Goal_received_home[k-1])  
                Goal_scored_away.append(Goal_scored_away[k-1])
                Goal_received_away.append(Goal_received_away[k-1])
                total_goal_scored.append(total_goal_scored[k-1]+ALL_TEAM[team_index].df['FTHG'][i])
                total_goal_received.append(total_goal_received[k-1]+ALL_TEAM[team_index].df['FTAG'][i])

                #Goal average
                Home=Home+1
                total=Home+Away
                Av_Goal_scored_home.append(float(Goal_scored_home[k])/Home)
                Av_Goal_received_home.append(float(Goal_received_home[k])/Home)
                Av_Goal_scored_away.append(Av_Goal_scored_away[k-1])
                Av_Goal_received_away.append(Av_Goal_received_away[k-1])
                Av_total_goal_scored.append(float(total_goal_scored[k])/total)
                Av_total_goal_received.append(float(total_goal_received[k])/total)


                #Point winning
                if ALL_TEAM[team_index].df['FTR'][i]=='H':
                    Point_home.append(3+Point_home[k-1])
                    Av_Point_home.append(float(Point_home[k])/Home)
                    total_point.append(3+total_point[k-1])
                    Av_total_point.append(float(total_point[k])/total)

                #Point draw
                if ALL_TEAM[team_index].df['FTR'][i]=='D':
                    Point_home.append(1+Point_home[k-1])
                    Av_Point_home.append(float(Point_home[k])/Home)
                    total_point.append(1+total_point[k-1])
                    Av_total_point.append(float(total_point[k])/total)

                #Point lost
                if ALL_TEAM[team_index].df['FTR'][i]=='A':
                    Point_home.append(0+Point_home[k-1])
                    Av_Point_home.append(float(Point_home[k])/Home)   
                    total_point.append(0+total_point[k-1])
                    Av_total_point.append(float(total_point[k])/total)

                Point_away.append(Point_away[k-1])
                Av_Point_away.append(Av_Point_away[k-1])

            #AWAY
            if ALL_TEAM[team_index].df['HomeTeam'][i]!=team_lists[team_index]:

                #Goal scored, received and total
                Goal_scored_away.append(ALL_TEAM[team_index].df['FTAG'][i]+Goal_scored_away[k-1])
                Goal_received_away.append(ALL_TEAM[team_index].df['FTHG'][i]+Goal_received_away[k-1])
                Goal_scored_home.append(Goal_scored_home[k-1])
                Goal_received_home.append(Goal_received_home[k-1])
                total_goal_scored.append(total_goal_scored[k-1]+ALL_TEAM[team_index].df['FTAG'][i])
                total_goal_received.append(total_goal_received[k-1]+ALL_TEAM[team_index].df['FTHG'][i])

                #Goal average
                Away=Away+1
                total=Home+Away

                Av_Goal_scored_away.append(float(Goal_scored_away[k])/Away)
                Av_Goal_received_away.append(float(Goal_received_away[k])/Away)
                Av_Goal_scored_home.append(Av_Goal_scored_home[k-1])
                Av_Goal_received_home.append(Av_Goal_received_home[k-1])
                Av_total_goal_scored.append(float(total_goal_scored[k])/total)
                Av_total_goal_received.append(float(total_goal_received[k])/total)


                #Point winning
                if ALL_TEAM[team_index].df['FTR'][i]=='A':
                    Point_away.append(3+Point_away[k-1])
                    Av_Point_away.append(float(Point_away[k])/Away)
                    total_point.append(3+total_point[k-1])
                    Av_total_point.append(float(total_point[k])/total)

                #Point draw
                if ALL_TEAM[team_index].df['FTR'][i]=='D':
                    Point_away.append(1+Point_away[k-1])
                    Av_Point_away.append(float(Point_away[k])/Away)

                    total_point.append(1+total_point[k-1])
                    Av_total_point.append(float(total_point[k])/total)

                #Point lost
                if ALL_TEAM[team_index].df['FTR'][i]=='H':
                    Point_away.append(0+Point_away[k-1])
                    Av_Point_away.append(float(Point_away[k])/Away)

                    total_point.append(0+total_point[k-1])
                    Av_total_point.append(float(total_point[k])/total)


                Point_home.append(Point_home[k-1])
                Av_Point_home.append(Av_Point_home[k-1])
                
                


            k=k+1

        
        Goal_scored_home[-1]=0
        Goal_received_home[-1]=0
        Goal_scored_away[-1]=0
        Goal_received_away[-1]=0
        total_goal_scored[-1]=0
        total_goal_received[-1]=0


        Av_Goal_scored_home[-1]=0
        Av_Goal_received_home[-1]=0
        Av_Goal_scored_away[-1]=0
        Av_Goal_received_away[-1]=0
        Av_total_goal_scored[-1]=0
        Av_total_goal_received[-1]=0


        Point_home[-1]=0
        Point_away[-1]=0
        Av_Point_home[-1]=0
        Av_Point_away[-1]=0

        total_point[-1]=0
        Av_total_point[-1]=0
        
        
        T_L3M_Pnt[-1]=0
        T_L5M_Pnt[-1]=0
        

    del Goal_scored_home[-1]
    del Goal_received_home[-1]
    del Goal_scored_away[-1]
    del Goal_received_away[-1]
    del total_goal_scored[-1]
    del total_goal_received[-1]


    del Av_Goal_scored_home[-1]
    del Av_Goal_received_home[-1]
    del Av_Goal_scored_away[-1]
    del Av_Goal_received_away[-1]
    del Av_total_goal_scored[-1]
    del Av_total_goal_received[-1]


    del Point_home[-1]
    del Point_away[-1]
    del Av_Point_home[-1]
    del Av_Point_away[-1]

    del total_point[-1]
    del Av_total_point[-1]
    
    del T_L3M_Pnt[-1]
    del T_L5M_Pnt[-1]
    
    ALL_TEAM[team_index].df['Round']=round_index
    
    ALL_TEAM[team_index].df['T_L3M']=T_L3M_Pnt
    ALL_TEAM[team_index].df['T_L5M']=T_L5M_Pnt
    
    ALL_TEAM[team_index].df['GS_H']=Goal_scored_home
    ALL_TEAM[team_index].df['GS_A']=Goal_scored_away
    ALL_TEAM[team_index].df['GR_H']=Goal_received_home
    ALL_TEAM[team_index].df['GR_A']=Goal_received_away
    ALL_TEAM[team_index].df['TGS']=total_goal_scored
    ALL_TEAM[team_index].df['TGR']=total_goal_received
    
    
    ALL_TEAM[team_index].df['AGS_H']=Av_Goal_scored_home
    ALL_TEAM[team_index].df['AGS_A']=Av_Goal_scored_away
    ALL_TEAM[team_index].df['AGR_H']=Av_Goal_received_home
    ALL_TEAM[team_index].df['AGR_A']=Av_Goal_received_away
    ALL_TEAM[team_index].df['ATGS']=Av_total_goal_scored
    ALL_TEAM[team_index].df['ATGR']=Av_total_goal_received

    
    ALL_TEAM[team_index].df['Pnt_H']=Point_home
    ALL_TEAM[team_index].df['Pnt_A']=Point_away
    ALL_TEAM[team_index].df['TOTPT']=total_point

    
    ALL_TEAM[team_index].df['Av_Pnt_H']=Av_Point_home
    ALL_TEAM[team_index].df['Av_pnt_A']=Av_Point_away
    ALL_TEAM[team_index].df['Av_tot_pnt']=Av_total_point
    
    
    return ALL_TEAM[team_index].df


In [847]:
#calculate_for_team_i(allteamsresults,teams,40)

In [848]:
for i in range(len(teams)):
    allteamsresults[i].df=calculate_for_team_i(allteamsresults,teams,i)


/Users/nicolamacchitella/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:473: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/nicolamacchitella/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:475: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/nicolamacchitella/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:476: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

In [849]:
tmp=pd.DataFrame(columns=(
                          'H_ROUND',
    
                          'H_TP3',
                          'H_TP5',
                            
    
                          'H_GS_H', 
                          'H_GS_A', 
                          'H_GR_H', 
                          'H_GR_A', 
                          'H_TGS',
                          'H_TGR', 
                          
                          'H_AvGS_H' ,
                          'H_AvGS_A' ,
                          'H_AvGR_H',
                          'H_AvGR_A',
                          'H_Av_TGS',
                          'H_Av_TGR', 
                          
                          
                          'H_Pnt_H' ,
                          'H_Pnt_A',
                          'H_T_pnt',
                          'H_Av_Pnt_H',
                          'H_Av_pnt_A',
                          'H_Av_T_pnt',
                          
                          'A_ROUND',
    
                          'A_TP3',
                          'A_TP5',

                          'Aw_GS_H', 
                          'Aw_GS_A', 
                          'Aw_GR_H', 
                          'Aw_GR_A', 
                          'Aw_TGS',
                          'Aw_TGR', 
                          
                          'Aw_AvGS_H' ,
                          'Aw_AvGS_A' ,
                          'Aw_AvGR_H',
                          'Aw_AvGR_A',
                          'Aw_Av_TGS',
                          'Aw_Av_TGR', 
                          
                          
                          'Aw_Pnt_H' ,
                          'Aw_Pnt_A',
                          'Aw_T_pnt',
                          'Aw_Av_Pnt_H',
                          'Aw_Av_pnt_A',
                         
                          'Aw_Av_T_pnt',
                          
                          
                          
                         )
                )



In [850]:
for i in season.index:
    date=season['Date'][i]
    Home=season['HomeTeam'][i]
    Away=season['AwayTeam'][i]
    
    home_index=pd.Series(teams)[teams==Home].index[0]
    away_index=pd.Series(teams)[teams==Away].index[0]
    tobeappend=list(allteamsresults[home_index].df.loc[i].iloc[38:]) + list(allteamsresults[away_index].df.loc[i].iloc[38:])
    tmp.loc[i]=tobeappend

In [851]:
final=pd.merge(left=season, right=tmp, left_index=True, right_index=True)

In [852]:
final['DIFF_TP']=final['H_T_pnt']-final['Aw_T_pnt']
final['DIFF_HP-AP']=final['H_Pnt_H']-final['Aw_Pnt_A']

final['SUMM_HGSH+HGSH+AGRA']=final['H_GS_H']+final['Aw_GR_A']
final['SUMM_HGRH+AGSA']=final['H_GR_H']+final['Aw_GS_A']
final['SUMM_HTGS+ATGR']=final['H_TGS']+final['Aw_TGR']
final['SUMM_HTGR+ATGS']=final['H_TGR']+final['Aw_TGS']

final['DIFF_L3MP']=final['H_TP3']-final['A_TP3']
final['DIFF_L5MP']=final['H_TP5']-final['A_TP5']


final['DIFF_av_TP']=final['H_Av_T_pnt']-final['Aw_Av_T_pnt']
final['DIFF_av_HP-AP']=final['H_Av_Pnt_H']-final['Aw_Av_pnt_A']

final['SUMM_av_HGSH+AGRA']=final['H_AvGS_H']+final['Aw_AvGR_A']
final['SUMM_av_HGRH+AGSA']=final['H_AvGR_H']+final['Aw_AvGS_A']

final['SUMM_av_HTGS+ATGR']=final['H_Av_TGS']+final['Aw_Av_TGR']
final['SUMM_av_HTGR+ATGS']=final['H_Av_TGR']+final['Aw_Av_TGS']

final['Goal/No_Goal'] = ((final['FTHG']>0 ) * 1)*((final['FTAG']>0) * 1)
final['Over2.5/Under2.5'] = ((final['FTHG'] + final['FTAG']) >2)*1
final['Over1.5/Under1.5'] = ((final['FTHG'] + final['FTAG']) >1)*1


In [853]:
final.to_csv('../Data/from_sqlite_Data/Realfinal_before_name_change.csv')

In [932]:
final=pd.read_csv('../Data/from_sqlite_Data/Realfinal_before_name_change.csv')

# ELO

In [855]:
import io
import requests

k=0
for i in final.Date.unique():
    try:
        a=i.replace('00:00:00','')
        a=a.replace(' ','')
        url='http://api.clubelo.com/'+a
        s=requests.get(url).content
        c=pd.read_csv(io.StringIO(s.decode('utf-8')))
        saveto='../Data/elo/'+a
        c.to_csv(saveto)
    except:
        'EmptyDataError: No columns to parse from file'
        k=k+1
    


In [933]:
concat=[]
k=0
for i in final.Date.unique():    
    a=i.replace('00:00:00','')
    a=a.replace(' ','')
    concat.append(pd.read_csv('../Data/elo/'+a))




In [934]:
all_elo=pd.concat(concat)

In [935]:
all_elo.to_csv('../Data/elo/elo_totale')

# Looking for differences dataframe with elo

In [936]:
def printme(a,b):
    for i in range(len(a)):
        print (a[i])
        print (b[i])
    return

In [937]:
our_team=final.HomeTeam.unique()
elo_team=all_elo.Club.unique()
for i in our_team:
    if i in elo_team:
        our_team=np.delete(our_team,np.argwhere(our_team==i))
        elo_team=np.delete(elo_team,np.argwhere(elo_team==i))

guesses_from_elo=[]
final_real=[]
no_guess=[]
for i in our_team:  
    if len(difflib.get_close_matches(i,elo_team))>0:
        final_real.append(i)
        guesses_from_elo.append(list(difflib.get_close_matches(i,elo_team)))  
    if len(difflib.get_close_matches(i,elo_team))==0:
            no_guess.append(i)

In [938]:
#printme(final_real,guesses_from_elo)

In [939]:
actual=['Stoke City','Sporting Lokeren',
        'Amadora','Chievo Verona',
        'Sevilla FC','Club Brugge KV','Real Madrid CF',
        'Villarreal CF','SC Bastia', 'Bolton Wanderers', 'Wigan Athletic','Wolverhampton Wanderers']
new=['Stoke','Lokeren','Estrela Amadora','Chievo','Sevilla','Brugge',
     'Real Madrid','Villarreal','Bastia','Bolton','Wigan' , 'Wolves']

for i in range(len(new)):
    final.HomeTeam=final.HomeTeam.replace(actual[i],new[i])
    final.AwayTeam=final.AwayTeam.replace(actual[i],new[i])

In [940]:
our_team=final.AwayTeam.unique()
elo_team=all_elo.Club.unique()
for i in our_team:
    if i in elo_team:
        our_team=np.delete(our_team,np.argwhere(our_team==i))
        elo_team=np.delete(elo_team,np.argwhere(elo_team==i))
guesses_from_elo=[]
final_real=[]
no_guess=[]
for i in our_team: 
    if len(difflib.get_close_matches(i,elo_team))>0:
        final_real.append(i)
        guesses_from_elo.append(list(difflib.get_close_matches(i,elo_team)))
    if len(difflib.get_close_matches(i,elo_team))==0:
            no_guess.append(i)

In [941]:
printme(final_real,guesses_from_elo)

FC Basel
['Basel', 'FC Tiraspol']
FC Sion
['Sion']
FC Vaduz
['Vaduz', 'Cadiz']
Grasshopper Club Zürich
['Grasshoppers']
FC Luzern
['Luzern', 'Lautern']
BSC Young Boys
['Young Boys']
FC Aarau
['Aarau', 'Farul']
AC Bellinzona
['Bellinzona', 'Avellino']
P. Warszawa
['Polonia Warszawa']
LOSC Lille
['Lille']
AS Saint-Étienne
['Saint-Etienne']
Grenoble Foot 38
['Grenoble']
FC Nantes
['Nantes']
FC Lorient
['Lorient', 'Forest']
Lechia Gdańsk
['Lechia']
Odra Wodzisław
['Odra Wodzislaw']
SM Caen
['Caen']
OGC Nice
['Nice']
Jagiellonia Białystok
['Jagiellonia']
Toulouse FC
['Toulouse']
St. Mirren
['St Mirren', 'St Truiden']
Widzew Łódź
['Widzew']
Hamburger SV
['Hamburg']
Borussia Dortmund
['Dortmund']
AS Monaco
['Monaco']
Le Havre AC
['Le Havre']
Sporting Charleroi
['Charleroi', 'Sporting', 'Sporting Fingal']
KV Kortrijk
['Kortrijk']
Standard de Liège
['Standard']
RSC Anderlecht
['Anderlecht']
VfL Bochum
['Bochum']
TSG 1899 Hoffenheim
['Hoffenheim']
Hannover 96
['Hannover']
Le Mans FC
['Le Mans']


In [942]:
for i in range(len(guesses_from_elo)):
    final.HomeTeam=final.HomeTeam.replace(final_real[i],guesses_from_elo[i][0])
    final.AwayTeam=final.AwayTeam.replace(final_real[i],guesses_from_elo[i][0])

In [943]:
absent_final=['Neuch\xc3\xa2tel Xamax',
 'FC Z\xc3\xbcrich',
 'Lech Pozna\xc5\x84',
 'AS Nancy-Lorraine',
 'Heart of Midlothian',
 'FC Sochaux-Montb\xc3\xa9liard',
 'Stade Rennais FC',
 'Wis\xc5\x82a Krak\xc3\xb3w',
 '\xc5\x9al\xc4\x85sk Wroc\xc5\x82aw',
 'Girondins de Bordeaux',
 'Arka Gdynia',
 'Olympique Lyonnais',
 'Polonia Bytom',
 'Hamilton Academical FC',
 'FC Bayern Munich',
 'Paris Saint-Germain',
 'FC Energie Cottbus',
 'Inverness Caledonian Thistle',
 'GKS Be\xc5\x82chat\xc3\xb3w',
 'Ruch Chorz\xc3\xb3w',
 'Borussia M\xc3\xb6nchengladbach',
 'Olympique de Marseille',
 'Vit\xc3\xb3ria Guimar\xc3\xa3es',
 'SV Werder Bremen',
 'Hertha BSC Berlin',
 'Royal Excel Mouscron',
 'RAEC Mons',
 'Naval 1\xc2\xb0 de Maio',
 '1. FC K\xc3\xb6ln',
 'Roda JC Kerkrade',
 'N.E.C.',
 'AZ']

In [944]:
absent_elo=['Xamax',
 'Zuerich',
 'Lech',
 'Nancy',
 'Hearts',
 'Sochaux',
 'Rennes',
 'Wisla',
 'Slask',
 'Bordeaux',
 'Arka',
 'Lyon',
 'Bytom',
 'Hamilton',
 'Bayern',
 'Paris SG',
 'Cottbus',
 'Inverness',
 'Belchatow',
 'Ruch',
 'Gladbach',
 'Marseille',
 'Guimaraes',
 'Werder',
 'Hertha',
 'Mouscron',
 'Bergen',
 'Naval',
 'Koeln',
 'Roda',
 'Nijmegen',
 'Alkmaar']

In [945]:
for i in range(len(absent_final)):
    final.HomeTeam=final.HomeTeam.replace(absent_final[i],absent_elo[i])
    final.AwayTeam=final.AwayTeam.replace(absent_final[i],absent_elo[i])

    

In [946]:
our_team=final.HomeTeam.unique()
elo_team=all_elo.Club.unique()

for i in our_team:
    if i in elo_team:
        our_team=np.delete(our_team,np.argwhere(our_team==i))
        elo_team=np.delete(elo_team,np.argwhere(elo_team==i))

In [947]:
our_team=final.HomeTeam.unique()
elo_team=all_elo.Club.unique()

for i in our_team:
    if i in elo_team:
        our_team=np.delete(our_team,np.argwhere(our_team==i))
        elo_team=np.delete(elo_team,np.argwhere(elo_team==i))

guesses_from_elo=[]
final_real=[]
no_guess=[]

for i in our_team:
    
    if len(difflib.get_close_matches(i,elo_team))>0:
        final_real.append(i)
        guesses_from_elo.append(list(difflib.get_close_matches(i,elo_team)))
       
    if len(difflib.get_close_matches(i,elo_team))==0:
            no_guess.append(i)

In [948]:
no=[]
for j in final.country_id.unique():
    a=final[final.country_id==j]
    #print('-----')
    #print (j)
    #print('-----')
    for i in no_guess:
        try:
            print( a[a.HomeTeam==i]['HomeTeam'].iloc[-1])
            no.append(i)
        except:
            'IndexError: single positional indexer is out-of-bounds'
            


Zagłębie Lubin
Pogoń Szczecin
Zawisza Bydgoszcz
Termalica Bruk-Bet Nieciecza
Partick Thistle F.C.
US Boulogne Cote D'Opale
Stade Brestois 29
Évian Thonon Gaillard FC
Stade de Reims
En Avant de Guingamp
1. FC Nürnberg
1. FSV Mainz 05
1. FC Kaiserslautern
SpVgg Greuther Fürth
Queens Park Rangers
Oud-Heverlee Leuven
Académica de Coimbra
Vitória Setúbal
"União de Leiria, SAD"
Reggio Calabria
RC Deportivo de La Coruña
Atlético Madrid
Athletic Club de Bilbao
Real Betis Balompié
Real Sporting de Gijón
Xerez Club Deportivo
Hércules Club de Fútbol
RC Celta de Vigo


In [949]:
elo_real_2=['Lubin', 'Pogon', 'Zawisza','Nieciecza','Partick','Boulogne','Brest','Evian TG','Reims','Guingamp','Nuernberg','Mainz','Karlsruhe','Fuerth','QPR','Leuven','Academica','Setubal','Leiria','Reggina','Depor','Atletico', 'Bilbao', 'Betis', 'Gijon','Xerez','Hercules','Celta']


In [950]:
for i in range(len(no)):
    final.HomeTeam=final.HomeTeam.replace(no[i],elo_real_2[i])
    final.AwayTeam=final.AwayTeam.replace(no[i],elo_real_2[i])

In [951]:
for i in final.HomeTeam.unique():
    if i not in all_elo.Club.unique():
        print(i)

In [952]:
for i in final.AwayTeam.unique():
    if i not in all_elo.Club.unique():
        print(i)

In [967]:
final.to_csv('../Data/from_sqlite_Data/Realfinal.csv')

# Merging with Dataframe

In [968]:
all_elo['From']=pd.to_datetime(all_elo['From'])
all_elo['To']=pd.to_datetime(all_elo['To'])

In [955]:
final.country_id.unique()

array(['Switzerland Super League', 'Poland Ekstraklasa',
       'Scotland Premier League', 'France Ligue 1',
       'Germany 1. Bundesliga', 'England Premier League',
       'Belgium Jupiler League', 'Portugal Liga ZON Sagres',
       'Netherlands Eredivisie', 'Italy Serie A', 'Spain LIGA BBVA'], dtype=object)

In [969]:
#final=final[final.country_id!='Poland Ekstraklasa']
#final.reset_index(inplace=True)

In [970]:
elo_teams_dataframes=[]
new_teams=pd.Series(final.HomeTeam.unique())
for i in range(len(new_teams)):
    elo_teams_dataframes.append(MyDf(all_elo[all_elo.Club==new_teams[i]]))

In [980]:
lis_home=[]
lis_away=[]
k=0
for j in range(len(final)):
    try:
        i=final.HomeTeam[j]
        z=final.AwayTeam[j]
        lis_home.append(elo_teams_dataframes[new_teams[new_teams==i].index[0]].df[elo_teams_dataframes[new_teams[new_teams==i].index[0]].df['To']>=final.Date[j]].iloc[0]['Elo'])
        lis_away.append(elo_teams_dataframes[new_teams[new_teams==z].index[0]].df[elo_teams_dataframes[new_teams[new_teams==z].index[0]].df['To']>=final.Date[j]].iloc[0]['Elo'])
    except:
        'IndexError: single positional indexer is out-of-bounds'
        #print(final.HomeTeam[j])
        #print(final.AwayTeam[j])
        k=k+1
        final.drop(j,inplace=True)

In [991]:
final['Home_elo']=lis_home
final['Away_elo']=lis_away
final['DIFF_ELO']=final['Home_elo']-final['Away_elo']


In [992]:
final.to_csv('../Data/from_sqlite_Data/Realfinal_withELO.csv')

In [988]:
final=pd.read_csv('../Data/from_sqlite_Data/Realfinal_withELO.csv')

In [989]:
final.drop(['Unnamed: 0'], axis=1,inplace=True)

In [990]:
final

,Unnamed: 0.1,index,Unnamed: 0.1.1,country_id,season,Date,HomeTeam,AwayTeam,FTHG,FTAG,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA,FTR,H_ROUND,H_TP3,H_TP5,H_GS_H,H_GS_A,H_GR_H,H_GR_A,H_TGS,H_TGR,H_AvGS_H,H_AvGS_A,H_AvGR_H,H_AvGR_A,H_Av_TGS,H_Av_TGR,H_Pnt_H,H_Pnt_A,H_T_pnt,H_Av_Pnt_H,H_Av_pnt_A,H_Av_T_pnt,A_ROUND,A_TP3,A_TP5,Aw_GS_H,Aw_GS_A,Aw_GR_H,Aw_GR_A,Aw_TGS,Aw_TGR,Aw_AvGS_H,Aw_AvGS_A,Aw_AvGR_H,Aw_AvGR_A,Aw_Av_TGS,Aw_Av_TGR,Aw_Pnt_H,Aw_Pnt_A,Aw_T_pnt,Aw_Av_Pnt_H,Aw_Av_pnt_A,Aw_Av_T_pnt,DIFF_TP,DIFF_HP-AP,SUMM_HGSH+HGSH+AGRA,SUMM_HGRH+AGSA,SUMM_HTGS+ATGR,SUMM_HTGR+ATGS,DIFF_L3MP,DIFF_L5MP,DIFF_av_TP,DIFF_av_HP-AP,SUMM_av_HGSH+AGRA,SUMM_av_HGRH+AGSA,SUMM_av_HTGS+ATGR,SUMM_av_HTGR+ATGS,Goal/No_Goal,Over2.5/Under2.5,Over1.5/Under1.5,Home_elo,Away_elo
0,0,0,0,Switzerland Super League,2008/2009,2008-07-18 00:00:00,Young Boys,Basel,1,2,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,A,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,1,1,1557.639893,1628.157349
1,1,1,1,Switzerland Super League,2008/2009,2008-07-19 00:00:00,Aarau,Sion,3,1,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,H,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,1,1,1412.096802,1448.046021
2,2,2,2,Switzerland Super League,2008/2009,2008-07-20 00:00:00,Luzern,Vaduz,1,2,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,A,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,1,1,1393.629761,1123.770142
3,3,3,3,Switzerland Super League,2008/2009,2008-07-20 00:00:00,Xamax,Zuerich,1,2,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,A,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,1,1,1399.865845,1521.358887
4,4,4,4,Switzerland Super League,2008/2009,2008-07-23 00:00:00,Bellinzona,Xamax,1,2,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,<null>,A,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,2.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,2.0,1.000000,0.000000,2.000000,0.0

# For Betting !

In [7]:
a=pd.read_csv('../Data/B1.csv')
season=a


In [8]:
#take the columns you care

In [9]:
season=season.iloc[:,1:7]

In [10]:
len(season.HomeTeam.unique())

16

In [11]:
#add the round to be predicted with 0 in all other values

In [12]:
round_to_be_predict=pd.DataFrame(columns=season.columns)

In [13]:
date=pd.to_datetime('01/05/17')

In [14]:
round_to_be_predict.loc[0]=[date,'Brugge','Waregem',0,0,'D']
#round_to_be_predict.loc[1]=[date,'Stoke','West Ham',0,0,'D']
#round_to_be_predict.loc[2]=[date,'Sunderland','Bournemouth',0,0,'D']
#round_to_be_predict.loc[3]=[date,'West Brom','Leicester',0,0,'D']
#round_to_be_predict.loc[4]=[date,'Crystal Palace','Burnley',0,0,'D']
#round_to_be_predict.loc[5]=[date,'Man United','Swansea',0,0,'D']
#round_to_be_predict.loc[6]=[date,'Everton','Chelsea',0,0,'D']
#round_to_be_predict.loc[7]=[date,'Middlesbrough','Man City',0,0,'D']
#round_to_be_predict.loc[8]=[date,'Tottenham','Arsenal',0,0,'D']
#round_to_be_predict.loc[9]=[date,'Watford','Liverpool',0,0,'D']



In [17]:
season

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,29/07/16,Mechelen,Club Brugge,0,2,A
1,30/07/16,Charleroi,Waasland-Beveren,1,0,H
2,30/07/16,Mouscron,Anderlecht,1,2,A
3,30/07/16,St Truiden,Lokeren,1,0,H
4,30/07/16,Waregem,Eupen,3,0,H
5,31/07/16,Genk,Oostende,2,1,H
6,31/07/16,Kortrijk,Gent,1,1,D
7,31/07/16,Westerlo,Standard,2,2,D
8,05/08/16,Oostende,Club Brugge,1,0,H
9,06/08/16,Eupen,Mechelen,0,2,A


In [18]:
round_to_be_predict.Date=pd.to_datetime(round_to_be_predict.Date)

In [19]:
season=pd.concat([season,round_to_be_predict]).reset_index()

In [20]:
season.drop('index',axis=1,inplace=True)

In [21]:
season['season']=['2016/2017']*len(season)


In [22]:
teams=season.HomeTeam.unique()
allteamsresults=[]
for i in teams:
    path=pd.DataFrame()            
    path = season[(season["HomeTeam"] == i) | (season['AwayTeam'] == i )] 
    allteamsresults.append(MyDf(path))

In [23]:
for i in range(len(teams)):
    allteamsresults[i].df=calculate_for_team_i(allteamsresults,teams,i)


/Users/nicolamacchitella/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:473: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/nicolamacchitella/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:475: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/nicolamacchitella/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:476: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

In [24]:
tmp=pd.DataFrame(columns=(
                          'H_ROUND',
    
                          'H_TP3',
                          'H_TP5',
                            
    
                          'H_GS_H', 
                          'H_GS_A', 
                          'H_GR_H', 
                          'H_GR_A', 
                          'H_TGS',
                          'H_TGR', 
                          
                          'H_AvGS_H' ,
                          'H_AvGS_A' ,
                          'H_AvGR_H',
                          'H_AvGR_A',
                          'H_Av_TGS',
                          'H_Av_TGR', 
                          
                          
                          'H_Pnt_H' ,
                          'H_Pnt_A',
                          'H_T_pnt',
                          'H_Av_Pnt_H',
                          'H_Av_pnt_A',
                          'H_Av_T_pnt',
                          
                          'A_ROUND',
    
                          'A_TP3',
                          'A_TP5',

                          'Aw_GS_H', 
                          'Aw_GS_A', 
                          'Aw_GR_H', 
                          'Aw_GR_A', 
                          'Aw_TGS',
                          'Aw_TGR', 
                          
                          'Aw_AvGS_H' ,
                          'Aw_AvGS_A' ,
                          'Aw_AvGR_H',
                          'Aw_AvGR_A',
                          'Aw_Av_TGS',
                          'Aw_Av_TGR', 
                          
                          
                          'Aw_Pnt_H' ,
                          'Aw_Pnt_A',
                          'Aw_T_pnt',
                          'Aw_Av_Pnt_H',
                          'Aw_Av_pnt_A',
                         
                          'Aw_Av_T_pnt',
                          
                          
                          
                         )
                )



In [25]:
for i in season.index:
    date=season['Date'][i]
    Home=season['HomeTeam'][i]
    Away=season['AwayTeam'][i]
    
    home_index=pd.Series(teams)[teams==Home].index[0]
    away_index=pd.Series(teams)[teams==Away].index[0]
    tobeappend=list(allteamsresults[home_index].df.loc[i].iloc[7:]) + list(allteamsresults[away_index].df.loc[i].iloc[7:])
    tmp.loc[i]=tobeappend

In [26]:
final=pd.merge(left=season, right=tmp, left_index=True, right_index=True)


final['DIFF_TP']=final['H_T_pnt']-final['Aw_T_pnt']
final['DIFF_HP-AP']=final['H_Pnt_H']-final['Aw_Pnt_A']

final['SUMM_HGSH+HGSH+AGRA']=final['H_GS_H']+final['Aw_GR_A']
final['SUMM_HGRH+AGSA']=final['H_GR_H']+final['Aw_GS_A']
final['SUMM_HTGS+ATGR']=final['H_TGS']+final['Aw_TGR']
final['SUMM_HTGR+ATGS']=final['H_TGR']+final['Aw_TGS']

final['DIFF_L3MP']=final['H_TP3']-final['A_TP3']
final['DIFF_L5MP']=final['H_TP5']-final['A_TP5']


final['DIFF_av_TP']=final['H_Av_T_pnt']-final['Aw_Av_T_pnt']
final['DIFF_av_HP-AP']=final['H_Av_Pnt_H']-final['Aw_Av_pnt_A']

final['SUMM_av_HGSH+AGRA']=final['H_AvGS_H']+final['Aw_AvGR_A']
final['SUMM_av_HGRH+AGSA']=final['H_AvGR_H']+final['Aw_AvGS_A']

final['SUMM_av_HTGS+ATGR']=final['H_Av_TGS']+final['Aw_Av_TGR']
final['SUMM_av_HTGR+ATGS']=final['H_Av_TGR']+final['Aw_Av_TGS']

final['Goal/No_Goal'] = ((final['FTHG']>0 ) * 1)*((final['FTAG']>0) * 1)
final['Over2.5/Under2.5'] = ((final['FTHG'] + final['FTAG']) >2)*1
final['Over1.5/Under1.5'] = ((final['FTHG'] + final['FTAG']) >1)*1



In [27]:
final.to_csv('../Data/from_sqlite_Data/belgium.csv')

In [28]:
final

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,season,H_ROUND,H_TP3,H_TP5,H_GS_H,H_GS_A,H_GR_H,H_GR_A,H_TGS,H_TGR,H_AvGS_H,H_AvGS_A,H_AvGR_H,H_AvGR_A,H_Av_TGS,H_Av_TGR,H_Pnt_H,H_Pnt_A,H_T_pnt,H_Av_Pnt_H,H_Av_pnt_A,H_Av_T_pnt,A_ROUND,A_TP3,A_TP5,Aw_GS_H,Aw_GS_A,Aw_GR_H,Aw_GR_A,Aw_TGS,Aw_TGR,Aw_AvGS_H,Aw_AvGS_A,Aw_AvGR_H,Aw_AvGR_A,Aw_Av_TGS,Aw_Av_TGR,Aw_Pnt_H,Aw_Pnt_A,Aw_T_pnt,Aw_Av_Pnt_H,Aw_Av_pnt_A,Aw_Av_T_pnt,DIFF_TP,DIFF_HP-AP,SUMM_HGSH+HGSH+AGRA,SUMM_HGRH+AGSA,SUMM_HTGS+ATGR,SUMM_HTGR+ATGS,DIFF_L3MP,DIFF_L5MP,DIFF_av_TP,DIFF_av_HP-AP,SUMM_av_HGSH+AGRA,SUMM_av_HGRH+AGSA,SUMM_av_HTGS+ATGR,SUMM_av_HTGR+ATGS,Goal/No_Goal,Over2.5/Under2.5,Over1.5/Under1.5
0,29/07/16,Mechelen,Club Brugge,0.0,2.0,A,2016/2017,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,1
1,30/07/16,Charleroi,Waasland-Beveren,1.0,0.0,H,2016/2017,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0
2,30/07/16,Mouscron,Anderlecht,1.0,2.0,A,2016/2017,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,1,1
3,30/07/16,St Truiden,Lokeren,1.0,0.0,H,2016/2017,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0
4,30/07/16,Waregem,Eupen,3.0,0.0,H,2016/2017,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,1
5,31/07/16,Genk,Oostende,2.0,1.0,H,2016/2017,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,1,1
6,31/07/16,Kortrijk,Gent,1.0,1.0,D,2016/2017,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,0,1
7,31/07/16,Westerlo,Standard,2.0,2.0,D,2016/2017,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,1,1
8,05/08/16,Oostende,Club Brugge,1.0,0.0,H,2016/2017,2.0,0.0,0.0,0.0,1.0,0.0,2.0,1.0,2.0,0.000000,1.000000,0.000000,2.000000,1.000000,2.000000,0.0,0.0,0.